## Предсказание законтрактованности путём агрегации данных о предыдущих поставках

Мы проведём сравнение двух методов:
1) Усреднение параметров поставок (таких как процент опоздания по поставкам и изменение качества)
2) Усреднение эмбеддингов предыдущих поставок, полученных через LSTM автоэнкодер

In [2]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score, average_precision_score, confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
import os
sys.path.append(os.path.abspath('..'))
from data.preprocessing import Preprocessor
from figures.plots import show_embeddings

import warnings
warnings.filterwarnings('ignore')

In [21]:
# таблица со спецификациями
spec = pd.read_csv('../data/processed_data/specs.csv')
spec['spec_date'] = pd.to_datetime(spec['spec_date'], format='%Y-%m-%d')
spec['delivery_period_end'] = pd.to_datetime(spec['delivery_period_end'], format='%Y-%m-%d')

# таблица с доставками
zpp4 = pd.read_csv('../data/processed_data/zpp4.csv')
zpp4['date'] = pd.to_datetime(zpp4['date'], format='%Y-%m-%d')
zpp4['spec_date'] = pd.to_datetime(zpp4['spec_date'], format='%Y-%m-%d')

In [22]:
spec.head(2)

,supplier,supplier_status,item,basis,spec_date,delivery_period_end,payment_terms,option,logistics,declared_price,consent_price,spec_price,volume_requested,volume_contracted,bids_submitted,bids_contracted,id,late
0,50 ЛЕТ ОКТЯБРЯ ООО,СХТП,подсолнечник,"Юг Руси, АО",2022-12-19,2022-12-25,По факту,15.0,0.0,25.5,25.0,25.0,40.0,60.0,1.0,1.0,"50 ЛЕТ ОКТЯБРЯ ООО_Юг Руси, АО_2022-12-19",0
1,50 ЛЕТ ОКТЯБРЯ ООО,СХТП,подсолнечник,"Юг Руси, АО",2023-01-24,2023-02-12,По факту,15.0,0.0,27.5,27.5,27.5,50.0,59.0,1.0,1.0,"50 ЛЕТ ОКТЯБРЯ ООО_Юг Руси, АО_2023-01-24",0


In [23]:
zpp4.head(2)

,date,supplier,basis,spec_date,contract_price,estimated_price,contract_price1,estimated_price1,quantity,sum,price_change,id
0,2022-07-02,СОЮЗ ООО Ростов,"Юг Руси, АО",2022-06-15,30.0,30.29,30.0,30.32,27.05,820156.0,1.07,"СОЮЗ ООО Ростов_Юг Руси, АО_2022-06-15"
1,2022-07-02,СОЮЗ ООО Ростов,"Юг Руси, АО",2022-06-15,30.0,30.29,30.0,31.02,28.78,892755.6,3.40,"СОЮЗ ООО Ростов_Юг Руси, АО_2022-06-15"


In [24]:
# средний процент на который опоздал поставщик
# средний процент привезённого товара
# средний процент ухудшения качества

In [25]:
# для вычисления переменных необходимы данные о спецификации,
# такие как дата окончания спецификации и объём поставок
zpp4 = pd.merge(zpp4, spec[['id', 'delivery_period_end', 'volume_contracted']], how='inner', on=['id', 'id'], suffixes = ('', '_DROP'))
zpp4.head(2)

,date,supplier,basis,spec_date,contract_price,estimated_price,contract_price1,estimated_price1,quantity,sum,price_change,id,delivery_period_end,volume_contracted
0,2022-07-06,Рязанова А.В. ИП Глава К(Ф)Х,"ЮР Валуйский МЭЗ ф-л, ООО",2022-07-06,25.0,24.78,25.0,24.78,27.44,679963.2,-0.88,Рязанова А.В. ИП Глава К(Ф)Х_ЮР Валуйский МЭЗ ...,2022-07-09,30.0
1,2022-07-12,Осипов Л.А. ИП Глава К(Ф)Х,"Юг Руси, АО",2022-07-12,25.0,24.68,25.0,24.55,31.43,771606.5,-1.80,"Осипов Л.А. ИП Глава К(Ф)Х_Юг Руси, АО_2022-07-12",2022-07-22,170.0


In [58]:
def lateness_percentage(row):
    if ((row['delivery_period_end'] - row['spec_date']).days > 0):
        # сколько прошло дней с начала спецификации
        days_passed = max((row['date']-row['spec_date']).days, 0)
        
        # сколько дней длится спецификация
        delivery_lenght = (row['delivery_period_end'] - row['spec_date']).days

    # Если спецификация началась и закончилась в один день,
    # то считаем длительность поставки равной одному дню,
    # а к количеству прошедших дней прибавляем 1.
    # Таким образом мы считаем, что спецификация начинается в начале указаного дня,
    # а доставка и конец спецификации в конце указанного дня.
    else:
        days_passed = max((row['date']-row['spec_date']).days + 1, 0)
        delivery_lenght = 1
    
    return days_passed / delivery_lenght

# На сколько поставщик близок к концу спецификации
zpp4['lateness_percentage'] = zpp4.apply(lambda row: lateness_percentage(row), axis=1)

# Доля уже доставленного товара от всего законтрактованого
zpp4['weight_percentage'] = zpp4.groupby('id')['quantity'].cumsum() / zpp4['volume_contracted']

# удалить записи с нулевым volume_contracted
zpp4 = zpp4.drop(zpp4.loc[zpp4['volume_contracted'] == 0].index)

# # дата последней поставки
# zpp4['last_date'] = zpp4.groupby(['id'])['date'].transform('max')

In [69]:
zpp4.head(2)

,date,supplier,basis,spec_date,contract_price,estimated_price,contract_price1,estimated_price1,quantity,sum,price_change,id,delivery_period_end,volume_contracted,lateness_percentage,weight_percentage
0,2022-07-06,Рязанова А.В. ИП Глава К(Ф)Х,"ЮР Валуйский МЭЗ ф-л, ООО",2022-07-06,25.0,24.78,25.0,24.78,27.44,679963.2,-0.88,Рязанова А.В. ИП Глава К(Ф)Х_ЮР Валуйский МЭЗ ...,2022-07-09,30.0,0.0,0.914667
1,2022-07-12,Осипов Л.А. ИП Глава К(Ф)Х,"Юг Руси, АО",2022-07-12,25.0,24.68,25.0,24.55,31.43,771606.5,-1.80,"Осипов Л.А. ИП Глава К(Ф)Х_Юг Руси, АО_2022-07-12",2022-07-22,170.0,0.0,0.184882


In [ ]:
# spec.apply(zpp4.loc[(zpp4['supplier'] == row['supplier']) & (zpp4['delivery_period_end'] < row['spec_date'])] \
#            .groupby('id').agg())

In [122]:
zpp4_agg = zpp4.groupby('id').agg({
    'supplier': lambda x: x.iloc[0],
    'delivery_period_end': lambda x: x.iloc[0],
    'lateness_percentage': lambda x: int(x.max() > 1),
    'weight_percentage': lambda x: int(x.max() < 1),
    'price_change': 'mean'
}).rename(columns={
    'lateness_percentage': 'lateness',
    'weight_percentage': 'underweight'
})

In [116]:
spec.apply(lambda row: zpp4_agg.loc[(zpp4_agg['supplier'] == row['supplier']) & (zpp4_agg['delivery_period_end'] < row['spec_date'])] \
           [['lateness', 'underweight', 'price_change']].mean(), axis=1)

In [123]:
spec[['lateness', 'underweight', 'price_change']] = spec.apply(lambda row: zpp4_agg.loc[(zpp4_agg['supplier'] == row['supplier']) & (zpp4_agg['delivery_period_end'] < row['spec_date'])] \
                                                               [['lateness', 'underweight', 'price_change']].mean(), axis=1)